In [1]:
import pandas as pd

In [2]:
media = pd.read_hdf('/mnt/gbif/clean_data.h5', 'media_merged_filtered-by-species_350pt')

In [3]:
from parallel_download import download

In [4]:
import mimetypes as mt

In [5]:
mt.add_type('image/pjpeg', '.jpg')

In [6]:
media['extension'] = media['format'].map(lambda x: mt.guess_extension(x, strict=False))

In [7]:
media['extension'].value_counts(dropna=False)

.jpg    663805
.png      1991
.gif         7
Name: extension, dtype: int64

In [8]:
len(media)

665803

In [22]:
import os
import io
import sys
import PIL


errors = []

for index, row in media.iterrows():
    ext = row['extension']
    filename = f'/mnt/gbif/media/{index}{ext}'
        
    try:
        PIL.Image.open(filename).verify()
    except PIL.Image.UnidentifiedImageError as err:
        print(f'{filename} - {index}')
        errors.append(index)
        # os.rename(filename, f'{stem}.uie{extension}')

errors

FileNotFoundError: [Errno 2] No such file or directory: '/mnt/gbif/media/0.jpg'

In [10]:
media.head()

,gbifID,format,identifier_media,identifier_obs,eventDate,decimalLatitude,decimalLongitude,taxonID,taxonRank,taxonomicStatus,...,classKey,orderKey,familyKey,genusKey,speciesKey,species,acceptedScientificName,verbatimScientificName,media_count_per_taxonID,extension
0,891021265,image/jpeg,https://inaturalist-open-data.s3.amazonaws.com...,119647,2012-08-31 00:00:00,38.036174,-122.755828,147221,SPECIES,ACCEPTED,...,180.0,1048.0,4812.0,8360956.0,2608086.0,Niebla homalea,Niebla homalea (Ach.) Rundel & Bowler,Niebla homalea,865,.jpg
5,891026285,image/jpeg,https://inaturalist-open-data.s3.amazonaws.com...,344410,2013-07-23 09:36:00,38.582730,-122.696660,54134,SPECIES,ACCEPTED,...,186.0,1145.0,3286.0,2519084.0,2548311.0,Trametes versicolor,Trametes versicolor (L.) Lloyd,Trametes versicolor,43873,.jpg
6,891026330,image/jpeg,https://inaturalist-open-data.s3.amazonaws.com...,345889,2013-07-25 19:33:13,43.627681,-72.529947,63274,SPECIES,ACCEPTED,...,186.0,1145.0,3290.0,2519220.0,7241318.0,Ganoderma tsugae,Ganoderma tsugae Murrill,Ganoderma tsugae,6970,.jpg
7,891027410,image/jpeg,https://inaturalist-open-data.s3.amazonaws.com...,381407,2013-08-19 11:10:07,40.499055,-121.402420,54608,SPECIES,ACCEPTED,...,180.0,1048.0,8305.0,2605402.0,2605405.0,Letharia columbiana,Letharia columbiana (Nutt.) J.W.Thomson,Letharia columbiana,2011,.jpg
8,891027410,image/jpeg,https://inaturalist-open-data.s3.amazonaws.com...,381407,2013-08-19 11:10:07,40.499055,-121.402420,54608,SPECIES,ACCEPTED,...,180.0,1048.0,8305.0,2605402.0,2605405.0,Letharia columbiana,Letharia columbiana (Nutt.) J.W.Thomson,Letharia columbiana,2011,.jpg


In [13]:
dirs = media['acceptedScientificName'].unique().tolist()

In [18]:
for directory in dirs:
    directory = f'/mnt/gbif/media/{directory}'
    print(directory)
    os.makedirs(directory)

/mnt/gbif/media/Niebla homalea (Ach.) Rundel & Bowler
/mnt/gbif/media/Trametes versicolor (L.) Lloyd
/mnt/gbif/media/Ganoderma tsugae Murrill
/mnt/gbif/media/Letharia columbiana (Nutt.) J.W.Thomson
/mnt/gbif/media/Amanita muscaria (L.) Lam.
/mnt/gbif/media/Tremella mesenterica Retz.
/mnt/gbif/media/Diploschistes muscorum (Scop.) R.Sant.
/mnt/gbif/media/Peltigera membranacea (Ach.) Nyl.
/mnt/gbif/media/Letharia vulpina (L.) Hue
/mnt/gbif/media/Ramalina menziesii Taylor
/mnt/gbif/media/Omphalotus olivascens H.E.Bigelow, O.K.Mill. & Thiers
/mnt/gbif/media/Pholiota squarrosa (Vahl) P.Kumm.
/mnt/gbif/media/Tubaria furfuracea (Pers.) Gillet
/mnt/gbif/media/Sphaerobolus stellatus Tode
/mnt/gbif/media/Crucibulum laeve (Huds.) Kambly
/mnt/gbif/media/Cantharellus californicus D.Arora & J.S.Dunham
/mnt/gbif/media/Gallowayella hasseana (Räsänen) S.Y.Kondr., Fedorenko, S.Stenroos, Kärnefelt, Elix, Hur & A.Thell
/mnt/gbif/media/Sticta limbata (Sm.) Ach.
/mnt/gbif/media/Bolbitius titubans (Bull.) Fr.

In [24]:
for index, row in media.iterrows():
    ext = row['extension']
    label = row['acceptedScientificName']
    filename = f'/mnt/gbif/media/{index}{ext}'
    new_filename = f'/mnt/gbif/media/{label}/{index}{ext}'
    try:
        os.rename(new_filename, filename)
    except FileNotFoundError:
        pass

In [25]:
for index, row in media.iterrows():
    ext = row['extension']
    label = row['acceptedScientificName']
    filename = f'/mnt/gbif/media/{index}{ext}'
    new_filename = f'/mnt/gbif/media/{label}/{index}{ext}'
    os.rename(filename, new_filename)
    